In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from xgboost import XGBClassifier

In [39]:
csv_path = "C:\\Users\\clombardi\\RL\\data.csv"
df = pd.read_csv(csv_path)
df

,ImagePath,FiberType,DateCreated,Diameter,CleaveAngle,CleaveTension,TensionVelocity,FHBOffset,ScribeDiameter,Misting,Hackle,CNN_Predicition,XGB_Prediction
0,Fiber-179Plus.png,PM15U25d,2025-06-09 15:37,123.5,0.22,193,60,2552.0,17.28,0,0.0,1,0.0
1,Fiber-183Plus.png,PM15U25d,2025-06-09 15:47,123.2,0.17,190,60,2552.0,16.70,0,0.0,1,0.0
2,Fiber-184Plus.png,PM15U25d,2025-06-09 15:50,123.4,0.33,191,60,2552.0,18.26,0,0.0,1,0.0
3,Fiber-185Plus.png,PM15U25d,2025-06-09 15:53,123.3,0.26,192,60,2552.0,20.61,0,0.0,1,0.0
4,Fiber-186Plus.png,PM15U25d,2025-06-09 15:56,123.9,0.38,193,60,2552.0,18.63,1,0.0,0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,Fiber-476Plus.png,FG250LA,2025-07-16 15:37,251.3,0.22,429,60,2520.0,29.65,0,0.0,1,0.0
202,Fiber-477Plus.png,FG250LA,2025-07-16 15:40,251.4,0.31,587,60,2487.0,31.08,0,0.0,1,0.0
203,Fiber-478Plus.png,FG250LA,2025-07-16 15:42,251.3,0.23,583,60,2487.0,24.87,0,0.0,0,0.0
204,Fiber-479Plus.png,FG250LA,2025-07-16 15:44,251.0,0.20,583,60,2487.0,30.00,0,0.0,1,0.0


In [40]:
encoded_df = pd.get_dummies(df, columns=['FiberType'], dtype=np.int32)
encoded_df

,ImagePath,DateCreated,Diameter,CleaveAngle,CleaveTension,TensionVelocity,FHBOffset,ScribeDiameter,Misting,Hackle,CNN_Predicition,XGB_Prediction,FiberType_400LA,FiberType_FG250LA,FiberType_PM15U25d
0,Fiber-179Plus.png,2025-06-09 15:37,123.5,0.22,193,60,2552.0,17.28,0,0.0,1,0.0,0,0,1
1,Fiber-183Plus.png,2025-06-09 15:47,123.2,0.17,190,60,2552.0,16.70,0,0.0,1,0.0,0,0,1
2,Fiber-184Plus.png,2025-06-09 15:50,123.4,0.33,191,60,2552.0,18.26,0,0.0,1,0.0,0,0,1
3,Fiber-185Plus.png,2025-06-09 15:53,123.3,0.26,192,60,2552.0,20.61,0,0.0,1,0.0,0,0,1
4,Fiber-186Plus.png,2025-06-09 15:56,123.9,0.38,193,60,2552.0,18.63,1,0.0,0,-1.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,Fiber-476Plus.png,2025-07-16 15:37,251.3,0.22,429,60,2520.0,29.65,0,0.0,1,0.0,0,1,0
202,Fiber-477Plus.png,2025-07-16 15:40,251.4,0.31,587,60,2487.0,31.08,0,0.0,1,0.0,0,1,0
203,Fiber-478Plus.png,2025-07-16 15:42,251.3,0.23,583,60,2487.0,24.87,0,0.0,0,0.0,0,1,0
204,Fiber-479Plus.png,2025-07-16 15:44,251.0,0.20,583,60,2487.0,30.00,0,0.0,1,0.0,0,1,0


In [55]:
low_range = 0.8
high_range = 1.4
unique_df = encoded_df.drop_duplicates(subset='ImagePath').copy()


synthetic_tensions = []
for img in unique_df['ImagePath']:
    tension = unique_df.loc[unique_df['ImagePath'] == img, 'CleaveTension']
    tensions = np.linspace(tension * low_range, tension * high_range, 50)
    synthetic_tensions.append(tensions)

In [56]:
synthetic_tensions

[array([[154.4       ],
        [156.76326531],
        [159.12653061],
        [161.48979592],
        [163.85306122],
        [166.21632653],
        [168.57959184],
        [170.94285714],
        [173.30612245],
        [175.66938776],
        [178.03265306],
        [180.39591837],
        [182.75918367],
        [185.12244898],
        [187.48571429],
        [189.84897959],
        [192.2122449 ],
        [194.5755102 ],
        [196.93877551],
        [199.30204082],
        [201.66530612],
        [204.02857143],
        [206.39183673],
        [208.75510204],
        [211.11836735],
        [213.48163265],
        [215.84489796],
        [218.20816327],
        [220.57142857],
        [222.93469388],
        [225.29795918],
        [227.66122449],
        [230.0244898 ],
        [232.3877551 ],
        [234.75102041],
        [237.11428571],
        [239.47755102],
        [241.84081633],
        [244.20408163],
        [246.56734694],
        [248.93061224],
        [251.293

In [57]:
def load_process_images(filename: str, set_mask: bool) -> "tf.Tensor":
    """Load and preprocess image from file path.

    Args:
        filename: Image filename or path

    Returns:
        tf.Tensor: Preprocessed image tensor
    """

    if tf is None:
        raise ImportError("TensorFlow is required for image processing")

    def load_image(file):
        """Load an image and process using same preprocessing as backbone.

        Args:
            file: path to image
            preprocess_input: processing from backbone model

        Returns:
            loaded and resized image
        """
        full_path = os.path.join(img_folder, file)

        try:
            img_raw = tf.io.read_file(full_path)
        except FileNotFoundError:
            print(f"Image file not found: {full_path}")
            return None
        except Exception as e:
            print(f"Error loading image {full_path}: {e}")
            return None

        try:
            img = tf.image.decode_png(img_raw, channels=1)
            img = tf.image.resize(img, [224, 224])
            img = tf.image.grayscale_to_rgb(img)
            if set_mask == True:
                img = mask_background(img)
            return img
        except Exception as e:
            print(f"Error processing image {full_path}: {e}")
            return None

    img = load_image(filename)
    img.set_shape([224, 224, 3])
    return img

In [50]:
def test_prediction(
        cnn_model,
        image_path: str,
        feature_vector,
        tension
    ) -> "np.ndarray":
        """Generate prediction for a single image (and features if not
        image_only).

        Args:
            image_path (str): Path to image to predict.
            feature_vector (np.ndarray | None): Numerical features (ignored if image_only).

        Returns:
            np.ndarray: Model prediction.
        """
        image = load_process_images(image_path, set_mask=False)
        image = np.expand_dims(image, axis=0)
        feature_vector = np.expand_dims(feature_vector, axis=0)
        feature_vector[0] = tension
        prediction = cnn_model.predict([image, feature_vector])
        return prediction

In [51]:
def gather_predictions(df, cnn_model, synthetic_tensions
    ) -> "tuple[np.ndarray, list, list] | tuple[None, None, None]":
        """Gather multiple predictions from test data.

        Returns:
            tuple: (true_labels, pred_labels, predictions) or (None, None, None) if no data.
        """
        if df is None:
            return None, None, None
        pred_image_paths = df["ImagePath"].values
        pred_features = df[
                [
                    "CleaveAngle",
                    "CleaveTension",
                    "ScribeDiameter",
                    "Misting",
                    "Hackle",
                ]
            ].values
        predictions = {}
        if pred_features is not None:
            i = 0
            surrogate_records = []
            for (img_path, feature_vector, tensions_for_scenario) in zip(
                pred_image_paths, pred_features, synthetic_tensions
            ):
                for tension in tensions_for_scenario:
                    prediction = test_prediction(cnn_model, img_path, feature_vector, tension)
                    record = {
                        "ImagePath": img_path,
                        "CleaveTension": tension,
                        "CNN_Prediction": prediction[0, 0]
                    }
                    surrogate_records.append(record)
            
        return surrogate_records

In [11]:
cnn_model_path = "G:\\Intern_Project_2025\\models\\Unified_classifier.keras"
try:
    cnn_model = tf.keras.models.load_model(cnn_model_path)
except ValueError:
    print("Model could not be loaded")

In [ ]:
import os

img_folder = "G:\\Intern_Project_2025\\combined_images"
predictions = gather_predictions(encoded_df, cnn_model, synthetic_tensions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━

In [59]:
new_df = pd.DataFrame(predictions)

In [60]:
new_df.to_csv("C:\\Users\\clombardi\\RL\\synthetic_data.csv")

In [70]:
len_fibers = len(df['FiberType'].unique())
fiber_types = encoded_df.iloc[:, -len_fibers:]
other_inputs = encoded_df[['CleaveTension', 'Diameter']]
total_inputs = pd.concat([other_inputs, fiber_types], axis=1)

In [71]:
from sklearn.model_selection import train_test_split

label_map = {-1:0, 0:1, 1: 2}
encoded_df['XGB_Prediction'] = encoded_df['XGB_Prediction'].map(label_map)

cnn_output = ['CNN_Predicition']
xgb_output = ['XGB_Prediction']

X = total_inputs
y_cnn = encoded_df[cnn_output]
y_xgb = encoded_df[xgb_output]

X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn = train_test_split(
    X, y_cnn, test_size = 0.2, random_state=42)
X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(
    X, y_xgb, test_size = 0.2, random_state=42)

In [121]:
xgb_predictor = XGBClassifier(
    objective="multi:softmax",
    num_classes=3,
    n_estimators=10000,
    max_depth=10,
    learning_rate=0.05,
    gamma=0.9,
    reg_lambda=0.9,
    reg_alpha=0.3,
    random_state=42)

xgb_predictor.fit(X_train_xgb, y_train_xgb)

C:\Users\clombardi\machine_learning\lib\site-packages\xgboost\training.py:183: UserWarning: [08:25:16] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "num_classes" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


,objective,'multi:softmax'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [122]:
print(f"XGB Surrogate Model Accuracy: {xgb_predictor.score(X_test_xgb, y_test_xgb):.2f}")

XGB Surrogate Model Accuracy: 0.79


In [146]:
cnn_predictor = XGBClassifier(
    objective="binary:logistic",
    n_estimators=1500,
    max_depth=3,
    learning_rate=0.01,
    random_state=42,
    reg_lamnda=0.9,
    reg_alpha=0.3,
)
cnn_predictor.fit(X_train_cnn, y_train_cnn)

C:\Users\clombardi\machine_learning\lib\site-packages\xgboost\training.py:183: UserWarning: [08:27:08] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "reg_lamnda" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [148]:
print(f"CNN Surrogate Model Accuracy: {cnn_predictor.score(X_test_cnn, y_test_cnn):.2f}")

CNN Surrogate Model Accuracy: 0.83


In [149]:
xgb_surrogate_path = "C:\\Users\\clombardi\\RL\\xgb_surrogate4.pkl"
cnn_surrogate_path = "C:\\Users\\clombardi\\RL\\cnn_surrogate4.pkl"

In [150]:
import joblib

joblib.dump(xgb_predictor, xgb_surrogate_path)
joblib.dump(cnn_predictor, cnn_surrogate_path)

['C:\\Users\\clombardi\\RL\\cnn_surrogate4.pkl']